<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Define-dataframes-and-function" data-toc-modified-id="Define-dataframes-and-function-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Define dataframes and function</a></span><ul class="toc-item"><li><span><a href="#print_row_detail-function" data-toc-modified-id="print_row_detail-function-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>print_row_detail function</a></span></li></ul></li><li><span><a href="#Test-and-refine-regex-patterns-for-flagging-likely-relevant-documents" data-toc-modified-id="Test-and-refine-regex-patterns-for-flagging-likely-relevant-documents-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test and refine regex patterns for flagging likely relevant documents</a></span><ul class="toc-item"><li><span><a href="#function:-check_regex_match" data-toc-modified-id="function:-check_regex_match-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>function: check_regex_match</a></span></li><li><span><a href="#Test-cases-for-regex" data-toc-modified-id="Test-cases-for-regex-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Test cases for regex</a></span></li><li><span><a href="#Working-list-for-flagging-positive-matches" data-toc-modified-id="Working-list-for-flagging-positive-matches-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Working list for flagging positive matches</a></span></li><li><span><a href="#Assign-initial-positive-and-negative-labels-using-regex" data-toc-modified-id="Assign-initial-positive-and-negative-labels-using-regex-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Assign initial positive and negative labels using regex</a></span><ul class="toc-item"><li><span><a href="#function:-label_with_regex" data-toc-modified-id="function:-label_with_regex-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>function: label_with_regex</a></span></li></ul></li><li><span><a href="#Create-lists-for-exploring-results-(to-be-replaced-by-using-DF)" data-toc-modified-id="Create-lists-for-exploring-results-(to-be-replaced-by-using-DF)-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Create lists for exploring results (to be replaced by using DF)</a></span></li><li><span><a href="#function:-print_docs_from_list" data-toc-modified-id="function:-print_docs_from_list-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>function: print_docs_from_list</a></span></li><li><span><a href="#Patterns-to-identify-paragraphs-with-few-or-no-numerals" data-toc-modified-id="Patterns-to-identify-paragraphs-with-few-or-no-numerals-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Patterns to identify paragraphs with few or no numerals</a></span></li><li><span><a href="#Check-for-false-positives" data-toc-modified-id="Check-for-false-positives-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Check for false positives</a></span></li></ul></li><li><span><a href="#Write-current-work-to-csv" data-toc-modified-id="Write-current-work-to-csv-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Write current work to csv</a></span></li></ul></div>

# Label paragraphs

This notebook labels paragraphs as "relevant" (positive hits) or not. 
Relevant = contains the employee count for the entire company for the most recent year. 

To do:
- Create reusable regex units, then compose regex using string join
- Modify print_row_detail function so that it can return the row index
    - Needed in order to create a list of false positives or negatives to correct

In [1]:
import numpy as np
import pandas as pd
import re

from bs4 import BeautifulSoup as bs

from path import Path, getcwdu

import glob
import os
from pathlib import PurePath
import copy

import random
import gzip
import shutil

## Define dataframes and function

In [2]:
full_path_list = [PurePath(os.getcwd()).joinpath(file).as_posix() for file in glob.iglob('../employee_filings/*.gz')]
full_file_list = [PurePath(file).name for file in glob.iglob('../employee_filings/*.gz')]
full_accession_ids = [PurePath(file).stem.replace('.html', '') for file in full_file_list]
full_cik_nbrs = [x.split(sep='-')[0] for x in full_accession_ids]

In [3]:
train_accession_ids = pd.read_csv('../data/train_accession_ids.csv', names=['acc_id'])['acc_id'].tolist()
val_accession_ids = pd.read_csv('../data/val_accession_ids.csv', names=['acc_id'])['acc_id'].tolist()

labeled_df = pd.read_excel('../data/train_val_employee_count_paragraphs.xlsx')
subset_df = pd.read_excel('../data/subset_employee_count_paragraphs.xlsx')

In [4]:
subset_file_list = [PurePath(os.getcwd()).joinpath('../employee_filings/').joinpath(file) for file in full_file_list if PurePath(file).stem.replace('.html', '') in subset_df.accession_number.unique().tolist()]

In [5]:
# After reading back in, the html needs to be parsed again
tbl_html_df = pd.read_csv('data/tbl_html_df.csv')
tbl_html_df['tbl_html'] = tbl_html_df.tbl_html.apply(lambda x: bs(x, 'lxml'))

In [ ]:
p_dtypes = {'acc_id' : 'category', 
'len' : 'int', 
'emp_header' : 'bool', 
'first_emp_head_block' : 'bool', 
'split' : 'category', 
'label' : 'uint8'}
paragraph_input_df = pd.read_csv('data/paragraph_input_df.csv', dtype=p_dtypes, index_col=0)

train_df = paragraph_input_df[paragraph_input_df.split == 'train']

val_df = paragraph_input_df[paragraph_input_df.split == 'val']

In [7]:
paragraph_input_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8205 entries, 0 to 8204
Data columns (total 9 columns):
acc_id                  8205 non-null category
para_text               8205 non-null object
len                     8205 non-null int32
emp_header              8205 non-null bool
first_emp_head_block    8205 non-null bool
para_text_orig          8205 non-null object
para_tag                8205 non-null object
split                   8205 non-null category
label                   8205 non-null uint8
dtypes: bool(2), category(2), int32(1), object(3), uint8(1)
memory usage: 353.7+ KB


### print_row_detail function

In [8]:
def print_row_detail(df=train_df, nrow=10, header_list = ['acc_id'],
                    detail_list = [ 'len', 'first_emp_head_block', 'split' ,'para_text'],
                    sortby=['acc_id', 'first_emp_head_block', 'len'], ascending=False):
    df_sorted = df.sort_values(sortby, ascending=ascending).reset_index()
    nrow = min(len(df_sorted), nrow)
    for i in range(0, nrow):
        for h in header_list:
            print('-'*35  + ' ' +  str(df_sorted[h][i]) + ' ' + '-'*35)
        for d in detail_list:
            print(d + '  :' + str(df_sorted[d][i]))
            print('')

In [9]:
print_row_detail()

----------------------------------- 0001703056-18-000008 -----------------------------------
len  :241

first_emp_head_block  :True

split  :train

para_text  :As of December 31, 2017, we employed approximately 18,000 people. Approximately 10% of our employees are covered by collective bargaining agreements. We believe that our relations with our employees and labor unions have generally been good.

----------------------------------- 0001703056-18-000008 -----------------------------------
len  :1249

first_emp_head_block  :False

split  :train

para_text  :James D. DeVries is our President. From May 2, 2016 to September 2017, Mr. DeVries served as The ADT Corporation's Executive Vice President and Chief Operating Officer. From December 2014 to May 2016, Mr. DeVries served as Executive Vice President of Brand Operations at Allstate Insurance Company, the second largest personal lines insurer in the United States. During his tenure at Allstate, Mr. DeVries led the operations organizati

## Test and refine regex patterns for flagging likely relevant documents

### function: check_regex_match

In [10]:
def check_regex_match(pattern, text_list):
    for idx, s in enumerate(text_list):
        mo = re.search(pattern, s)
        if mo:
            ms = mo.span()[1]
            print("------    " + str(idx) + "   Matched!    -----")
            print('str length  :' + str(len(s)) + '    match span  :' + str(ms))
            print(s[:ms])
            print('')
            print(s[ms:])
            print(re.search(pattern, s))
        else:
            print("------    " + str(idx) + "  NO MATCH    -----")
            print(s)

### Test cases for regex

In [102]:
test_list = ["We are a small company with approximately 61 employees.",
"The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015. "
"Total workforce level at December 31, 2016 was approximately 150,500",           
"Currently, the Company and its subsidiaries have an aggregate of 35 employees.",
"we employ only 31 employees", 
"We currently have 21 employees",
"((we|the compnay) )(currently )?((has|have|employ(s)?) )(the equivalent of )([0-9]{1,3},)*[0-9]{1,3}?"
"We currently employ 26 full-time employees",
"Including our full and part-time personnel, we estimate that we have \
             the equivalent of 12 full time employees."
"As a REIT, we employ only 31 employees and have a cost-effective \
             management structure."
            ]
print(len(test_list[4]))
check_regex_match(re.compile(r"((number of|head[ -]?count|total|workforce) )[^.]+ (employees|team members) [^.]+ ([0-9]{1,3},)*[0-9]{1,3}[^\d]", re.I),
                 test_list)

30
------    0  NO MATCH    -----
We are a small company with approximately 61 employees.
------    1   Matched!    -----
str length  :196    match span  :121
The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31,

 2015. Total workforce level at December 31, 2016 was approximately 150,500
<_sre.SRE_Match object; span=(4, 121), match='number of full-time employees of the Company was >
------    2  NO MATCH    -----
Currently, the Company and its subsidiaries have an aggregate of 35 employees.
------    3  NO MATCH    -----
we employ only 31 employees
------    4  NO MATCH    -----
We currently have 21 employees
------    5  NO MATCH    -----
((we|the compnay) )(currently )?((has|have|employ(s)?) )(the equivalent of )([0-9]{1,3},)*[0-9]{1,3}?We currently employ 26 full-time employees
------    6  NO MATCH    -----
Including our full and part-time personnel, we estimate that we have              the equivalent of 12 full 

### Working list for flagging positive matches

In [47]:
flag_pat_list = [
r"employ((ed|s)?) ((approximately|the equivalent of) )*([0-9]{1,3},)*[0-9]{1,3} ((permanent|full|time|part|time|temporary|total|and|equivalent)[ -])*(employees|individuals|people|team members|members|persons|associates)",
r"([0-9]{1,3},)*[0-9]{1,3} [^.]+ ((employ\w*|head([ -])?count|member(s)?|person(s)?|people|staff|team|workforce))",
r"((employ(ed)?|employee count|head([ -])?count|member(s)?|person(s)?|people|staff|team|workforce) )+[^.]+ *([0-9]{1,3},)*[0-9]{1,3}",
r"((number of|head[ -]?count|total|workforce) )[^.]+ (employees|team members) [^.]+ ([0-9]{1,3},)*[0-9]{1,3}[^\d]",
r"([0-9]{1,3},)*[0-9]{1,3} ((permanent|full|time|part|time|temporary|total|and|equivalent)[ -])*(employees|team members)",
r"(employees)(of the company )?((is|was|approximately) )*([0-9]{1,3},)*[0-9]{1,3}"]


#r"((permanent|full|part|time|full-time|part-time|temporary) )+(employees|team members|associates)",
#r"employed( approximately)?$", 
#r"Total workforce",

#r"^((permanent|full|part|time|full-time|part-time|temporary|total) )*(employees|team members|associates)"
                 
flag_pats = [re.compile(x, re.I) for x in flag_pat_list]

In [48]:
para_list = train_df.para_text.tolist()

val_para_list = val_df.para_text.tolist()

### Assign initial positive and negative labels using regex

#### function: label_with_regex

In [97]:

def label_with_regex(df, regex_list, text_col = 'para_text', label_col = 'label', 
                    reset_labels_first = True):
    """Label df in place if text matches any regex in regex_list"""
    df_para_list = df[text_col].tolist()
    pos_ids = set()
    for i, para in enumerate(df_para_list):
    ## "any() does not work for this purpose - it takes some explosive logic path"
    #if not any(reg.search(para) for reg in nonum_regs) and k2 not in para_dict[k]['df'].keys():
        for reg in regex_list:
            if reg.search(para):
                pos_ids.add(df.index[i])
                break
    if reset_labels_first:
        df.loc[:, label_col] = 0
    df.loc[df.index.isin(pos_ids), label_col] = 1

In [99]:
label_with_regex(val_df, flag_pats)

C:\Anaconda\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Create lists for exploring results (to be replaced by using DF)

In [110]:
train_df[np.logical_and(train_df.label ==1, 
                        train_df.len <= 21)][train_df.len >11].para_text.tolist()

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


['8.2 Employees.',
 '50 Employees; or',
 '4.24    Employees.',
 '13.1 Employees',
 '10 Employees',
 '8.3.18 Employees',
 '100 - 5,000 employees',
 '20 - 100 employees',
 '7.1.8 Employees.',
 '4.18 Employees.',
 '4.18 Employees.',
 'Note 27 Employees',
 '27 EMPLOYEES']

In [121]:
nonum_pat = re.compile(r"^([^\d]|[$][\d]+)[^\d]*$", re.I)
pos_key_list = train_df[(train_df.label ==1) & (train_df.len > 19)].acc_id.tolist()
pos_list = train_df[(train_df.label ==1) & (train_df.len > 19)].para_text.tolist()
neg_key_list = train_df[(train_df.label ==0) & (train_df.len > 19)].acc_id.tolist()
neg_list = train_df[np.logical_and(train_df.label ==0, train_df.len > 19)].para_text.tolist()
neg_key_list = [x for x in neg_key_list if not re.match(nonum_pat, neg_list[neg_key_list.index(x)])]
neg_list = [x for x in neg_list if not re.match(nonum_pat, x)]


In [122]:
print('Total paragraphs: ' + str(len(pos_list + neg_list)))

print('Positive paragraphs: ' + str(len(pos_list)))
print('Total keys: ' + str(len(train_df.acc_id.unique())))
print('Keys with positive labels: ' + str(len(train_df[train_df.label ==1].acc_id.unique())))
print('Negative paragraphs: ' + str(len(neg_list)))
print('Keys with negative labels: ' + str(len(train_df[train_df.label ==0].acc_id.unique())))

Total paragraphs: 4066
Positive paragraphs: 3428
Total keys: 1634
Keys with positive labels: 1535
Negative paragraphs: 638
Keys with negative labels: 1417


In [61]:
train_df[train_df.len == 10].head(20)

,acc_id,para_text,len,emp_header,first_emp_head_block,para_text_orig,para_tag,split,label
78,0000011544-17-000009,Employees,10,False,False,Employees,"<div class=""c240""><span class=""c239"">Employees...",train,0
139,0000018498-16-000065,Employees',10,False,False,Employees',"<div class=""c19"">Employees'</div>",train,0
141,0000018498-16-000065,Employees',10,False,False,Employees',"<div class=""c205"">Employees'</div>",train,0
142,0000018498-16-000065,Employees',10,False,False,Employees',"<div class=""c334"">Employees'</div>",train,0
489,0000068709-17-000026,Employees,10,False,False,Employees,"<div class=""c1""><span class=""c76"">Employees</s...",train,0
503,0000072207-17-000014,Employees,10,False,False,Employees,"<div class=""c172""><span class=""c56"">Employees<...",train,0
640,0000089089-17-000006,Associates,10,False,False,Associates,"<div class=""c67"">Associates</div>",train,0
808,0000105634-17-000043,Employees,10,False,False,Employees,"<div class=""c77""><span class=""c80""> </span><a ...",train,0
1033,0000717538-17-000060,Employees',10,False,False,Employees',"<div class=""c53"">Employees'</div>",train,0
1077,0000729580-17-000006,Associates,10,False,False,Associates,"<div class=""c78"">Associates</div>",train,0


### function: print_docs_from_list

In [37]:
def print_docs_from_list(key_list: list, doc_list: list, start: int=0, ndocs: int=50):
    end = start + ndocs
    for idx, tup in enumerate(zip(key_list[start:end], doc_list[start:end])):
        print(str(idx + start) + '   ------   ' + str(tup[0]))
        print(tup[1])

### Patterns to identify paragraphs with few or no numerals

These paragraphs are often not relevant

In [112]:
nonum_pats = [r"^([^\d]|[$][\d]+)[^\d]*$",
    r"^[^\d]{0,2}\d{1,2}[^\d]*$", 
    r"^[^\d]* \d{1,2}[^\d]*$", 
    r"^([^\d]+(\d{1,2}[^\d]{2,}){1,5}\d{1,3}[^\d]*)$",
    r"^((\d{1,3}[^\d]{2,}){1,4}\d{1,2}[^\d]*)$"]
year_and_num = re.compile(r"20[0-2][0-9].*[0-9]{1,3}.*|[0-9]{1,3}.*20[0-2][0-9]", re.I)
nonum_regs = [re.compile(x) for x in nonum_pats]

In [113]:
for y in nonum_pats:
    print(len([x for x in pos_list if re.search(y, x)]))

0
10
31
49
3


In [114]:
for y in nonum_pats:
    print(len([x for x in neg_list if re.search(y, x)]))

824
11
55
37
0


In [35]:
check_regex_match(nonum_pats[1], [x for x in pos_list if re.search(nonum_pats[1], x)])

------    0   Matched!    -----
str length  :1200    match span  :1200
We share facilities, personnel and other resources with several affiliated entities including, among others, Gould Investors L.P., a master limited partnership involved primarily in the ownership and operation of a diversified portfolio of real estate assets, and One Liberty Properties, Inc., an NYSE listed equity REIT. Eight individuals (including Jeffrey A. Gould, Chief Executive Officer and President, Mitchell Gould, Executive Vice President and George Zweier, Chief Financial Officer), devote substantially all of their business time to our activities, while our other personnel (including several officers) share their services on a part-time basis with us and other affiliated entities that share our executive offices. (Including our full and part-time personnel, we estimate that we have the equivalent of 12 full time employees. The allocation of expenses for the shared facilities, personnel and other resources is 

### Check for false positives

In [84]:
row_filter = np.logical_and(train_df.label == 1, 
    train_df.para_text.apply(lambda x: bool(re.search(nonum_pats[3], x)))
                           )

print_row_detail(df=train_df[row_filter], nrow=50, header_list = ['acc_id'],
                    detail_list = [ 'len', 'first_emp_head_block','para_text'],
                    sortby=['first_emp_head_block','acc_id',  'len'], ascending=False)

----------------------------------- 0001672764-17-000046 -----------------------------------
len  :203

first_emp_head_block  :False

para_text  :Currently our board has 11 members (see "Item 6. Directors, Senior Management and Employees-A. Directors and Senior Management-Board of Directors"). The board meets as often as required by our interests.

----------------------------------- 0001648416-17-000016 -----------------------------------
len  :1349

first_emp_head_block  :False

para_text  :Dutch legal requirements concerning director independence differ in certain respects from the rules applicable to U.S. companies listed on the NYSE. While under most circumstances both regimes require that a majority of board members be "independent," the definition of this term under Dutch law differs from the definition used under the NYSE corporate governance standards. In some cases the Dutch requirement is more stringent, such as by requiring a longer "look- back" period (five years) for form

This code will probably be replaced by the dataframe version, but the dataframe version needs to return the index first

In [60]:
print_docs_from_list(doc_list=[x for x in pos_list if re.search(nonum_pats[2], x)], 
    key_list=[pos_key_list[i] for i in [pos_list.index(x) for x in pos_list if re.search(nonum_pats[2], x)] ])

0   ------   0000014846-16-000062
We share facilities, personnel and other resources with several affiliated entities including, among others, Gould Investors L.P., a master limited partnership involved primarily in the ownership and operation of a diversified portfolio of real estate assets, and One Liberty Properties, Inc., an NYSE listed equity REIT. Eight individuals (including Jeffrey A. Gould, Chief Executive Officer and President, Mitchell Gould, Executive Vice President and George Zweier, Chief Financial Officer), devote substantially all of their business time to our activities, while our other personnel (including several officers) share their services on a part-time basis with us and other affiliated entities that share our executive offices. (Including our full and part-time personnel, we estimate that we have the equivalent of 12 full time employees. The allocation of expenses for the shared facilities, personnel and other resources is computed in accordance with a Shared 

In [105]:
print_docs_from_list()

0   ------   0000005272-17-000017
All full and part-time employees of the Company, excluding external contractors, independent contractors, temporary workers, and independent agents during the applicable Performance Year (the "Participants") are eligible to participate in this Plan for such Performance Year, unless the employee is a participant in another variable pay or sales plan that the applicable business has determined is in lieu of this Plan during such Performance Year.  For the avoidance of doubt, employees who are eligible to participate in a bonus plan that is required to be provided under local law or who have an employment contract with AIG or its subsidiaries for ongoing employment of unlimited duration that is not confined to a specific, finite project will not be ineligible for the Plan (unless the applicable business expressly elects to exclude such employee).  If an individual is hired after the Performance Year commences, the individual may become a Participant in th

## Write current work to csv

This allows code cleanup in this notebook while tuning classifiers in another notebook

In [101]:
train_df.to_csv('data/classifier_input_train.csv')
val_df.to_csv('data/classifier_input_val.csv')